In [ ]:
from twittercrawler.data_io import SocketReader

In [ ]:
reader = SocketReader(7000)

In [ ]:
records = []
for msg in reader.read(return_dict=True):
    records.append(msg)
    print(msg)

In [ ]:
reader.close()

In [ ]:
import pandas as pd
print(pd.DataFrame(records))

In [ ]:
import numpy as np
import pandas as pd
import os, shutil

In [ ]:
from twittercrawler.replies.components import SearchEngine, UserTweetStore
from twittercrawler.replies.query import TweetQuery
from twittercrawler.replies.collector import ReplyCollector

In [ ]:
tweet_id = "1347260116932976643"

In [ ]:
from twittercrawler.crawlers import RecursiveCrawler
crawler = RecursiveCrawler()
crawler.authenticate("../api_key.json")
status = crawler.twitter_api.show_status(id=tweet_id, tweet_mode="extended")
q = TweetQuery(status)
print(q)

store = UserTweetStore("alma")
store.update(q, q.since_id+10000000)
store.save()
store.user_intervals

store_tmp = UserTweetStore("alma")
store_tmp.load()
assert store_tmp.user_intervals == store.user_intervals
store_tmp.reset()

tweet_ids = [
    "1346181687600939009",#https://twitter.com/FogCityMidge/status/1346181687600939009
    "1346188735961038848",#https://twitter.com/ThePlumLineGS/status/1346188735961038848
    "1346241519179718656",#https://twitter.com/CNN/status/1346241519179718656
    "1346199928331251715",#https://twitter.com/jennafarmeruk/status/1346199928331251715
    "1346189795261886472",#https://twitter.com/BBCNews/status/1346189795261886472
    "1346149033279160323",#https://twitter.com/MattHancock/status/1346149033279160323
    "1346242405465530368",#https://twitter.com/TeaPainUSA/status/1346242405465530368 (new rumor)
    "1346106929140150273",#https://twitter.com/nbcwashington/status/1346106929140150273
    "1345927336345137152",#https://twitter.com/chipfranklin/status/1345927336345137152
]

# TODO

- include artificial max_id stopping after a few minutes for the crawler?
- if renew is on -> **update metrics** in the already exported thread as well!!!
- write a few tests...
- include comet logging...

In [ ]:
store_dir = "user_store"
collector_dir = "vaccine_collector"

if os.path.exists(store_dir):
    shutil.rmtree(store_dir)

shutil.rmtree(collector_dir)

In [ ]:
comet_info = None#("../comet_key.txt","collector_test","covid-vaccine")

In [ ]:
from twittercrawler.crawlers import RecursiveCrawler
crawler = RecursiveCrawler()
crawler.authenticate("../api_key_other.json")
store = UserTweetStore(store_dir)
engine = SearchEngine(crawler, store)

In [ ]:
len(store.user_intervals)

tweet_ids = [
    "1346615171482058754",#https://twitter.com/roccogalatilaw/status/1346615171482058754 (anti-vaxxer - 2021-01-06)
]

import time
for tweet_id in tweet_ids:
    collector = ReplyCollector(engine, tweet_id)
    collector.run()
    time.sleep(300)

In [ ]:
#https://twitter.com/hadleywickham/status/1347260116932976643
collector = ReplyCollector(engine, tweet_id, collector_dir, min_engagement=1, postpone_day_limit=0, action_day_limit=1)

store.reset()

- 26.7 s a kis tweetre a futásidő

import cProfile

cProfile.run("collector.run(comet_info=comet_info, feedback_interval=1)", sort="tottime")

In [ ]:
%%time
collector.run(comet_info=comet_info, feedback_interval=1)

In [ ]:
thread_df = pd.read_csv("vaccine_collector/%s/thread.csv" % tweet_id)
queue_df = pd.read_csv("vaccine_collector/%s/queue.csv" % tweet_id)
thread_df["id_str"] = thread_df["id_str"].astype("float64")
thread_df["user"] = thread_df["user"].apply(eval)

In [ ]:
thread_df.dtypes

### I should enforce types everywhere???

In [ ]:
#repliers = set()
edges = []
mentions = []
#thread_events = sorted(collector.tweet_thread, key=lambda x: x["id"])
#for tw in thread_events:
for _, tw in thread_df.iterrows():
    if tw["in_reply_to_status_id_str"] != None:
        edges.append((tw["id_str"],tw["in_reply_to_status_id_str"]))
        mentions.append((float(tw["user"]["id_str"]),tw["in_reply_to_user_id_str"]))
    #repliers.add(tw["user"]["screen_name"])
    #print(tw["user"]["screen_name"], tw["created_at"])
    #print(tw["full_text"])
    #print()

In [ ]:
import networkx as nx
G = nx.DiGraph()
H = nx.MultiDiGraph()
G.add_edges_from(edges)
_ = H.add_edges_from(mentions)

In [ ]:
nx.draw(G, node_size=10)

In [ ]:
nx.draw(H, node_size=10)

### covid vaccine min_replies:100 until:2021-01-05 since:2021-01-04